```{contents}
```
## Containerization

**Containerization** is the practice of packaging a LangGraph application together with its runtime environment, dependencies, configuration, and system tools into a **self-contained executable unit (container)**.
In production LangGraph systems, containerization is the foundation for **reliability, scalability, reproducibility, and safe deployment**.

---

### **1. Why LangGraph Requires Containerization**

LangGraph workflows are **stateful, multi-step, multi-agent systems** that depend on:

* Python runtime and libraries
* LLM SDKs and model clients
* External tools (databases, vector stores, APIs)
* State stores and memory backends
* Environment-specific configuration

Without containerization:

* Dependency drift breaks executions
* Scaling becomes unsafe
* Debugging is inconsistent
* Reproducibility is impossible

With containerization, every execution environment is **deterministic and portable**.

---

### **2. What Gets Containerized in a LangGraph System**

| Component            | Included in Container |
| -------------------- | --------------------- |
| LangGraph runtime    | ✔                     |
| Python application   | ✔                     |
| Graph definitions    | ✔                     |
| Node implementations | ✔                     |
| LLM SDKs             | ✔                     |
| Tool clients         | ✔                     |
| Configuration        | ✔                     |
| Monitoring agents    | ✔                     |
| Security policies    | ✔                     |

External services such as **Postgres, Redis, vector DBs, LLM APIs** remain outside the container and are accessed over the network.

---

### **3. Canonical Container Architecture**

```
Client → API Gateway → LangGraph Container → LLM / Tools / Memory
```

Each container instance executes graphs independently but shares **external state stores**.

---

### **4. Minimal Production Dockerfile**

```dockerfile
FROM python:3.10-slim

WORKDIR /app

COPY requirements.txt .
RUN pip install --no-cache-dir -r requirements.txt

COPY . .

ENV PYTHONUNBUFFERED=1

CMD ["python", "main.py"]
```

---

### **5. Example LangGraph App Structure**

```
/app
 ├─ main.py
 ├─ graph.py
 ├─ nodes/
 │   ├─ planner.py
 │   ├─ executor.py
 │   └─ tools.py
 ├─ state.py
 └─ requirements.txt
```

---

### **6. Containerized LangGraph Execution**

```bash
docker build -t langgraph-app .
docker run -p 8000:8000 --env-file .env langgraph-app
```

---

### **7. Scaling with Containers**

LangGraph graphs are **naturally parallelizable** at the container level.

| Layer       | Scaling Responsibility |
| ----------- | ---------------------- |
| Container   | Horizontal scaling     |
| State store | Shared persistence     |
| LLM API     | Elastic provider       |
| Task queue  | Load balancing         |

In Kubernetes:

```
Deployment → ReplicaSet → Pods (LangGraph containers)
```

---

### **8. Production Variants**

| Variant                          | Use Case           |
| -------------------------------- | ------------------ |
| Single container                 | Local dev          |
| Multi-container (Docker Compose) | Microservices      |
| Kubernetes                       | Enterprise / cloud |
| Serverless containers            | Burst workloads    |

---

### **9. Fault Tolerance & Recovery**

Containerization enables:

* Automatic restarts
* Crash isolation
* Safe rollbacks
* Canary deployments
* Blue–green upgrades

LangGraph state persistence ensures that **graph execution resumes safely** after container failure.

---

### **10. Security Benefits**

* Dependency isolation
* Secrets injection via environment
* Network sandboxing
* Controlled tool execution
* Immutable runtime

---

### **11. Mental Model**

A LangGraph container is a **deterministic execution engine** for agentic workflows:

> **Graph + State + Tools + Runtime → Immutable Execution Unit**

This guarantees that **what you test is what you deploy**.

### Demonstration

In [ ]:
# =========================
# 1️⃣ Create project files
# =========================

mkdir langgraph_demo && cd langgraph_demo

cat > requirements.txt <<EOF
langgraph
langchain
openai
EOF

cat > main.py <<EOF
from typing import TypedDict
from langgraph.graph import StateGraph, END

class State(TypedDict):
    text: str
    count: int

def process(state):
    print("Processing:", state)
    return {"count": state["count"] + 1}

def router(state):
    return END if state["count"] >= 3 else "process"

builder = StateGraph(State)
builder.add_node("process", process)
builder.set_entry_point("process")
builder.add_conditional_edges("process", router, {"process": "process", END: END})

graph = builder.compile()
result = graph.invoke({"text": "hello", "count": 0})
print("Final:", result)
EOF

cat > Dockerfile <<EOF
FROM python:3.10-slim
WORKDIR /app
COPY requirements.txt .
RUN pip install --no-cache-dir -r requirements.txt
COPY . .
CMD ["python", "main.py"]
EOF

# =========================
# 2️⃣ Build the container
# =========================

docker build -t langgraph-container .

# =========================
# 3️⃣ Run the container
# =========================

docker run --rm langgraph-container
